In [3]:
import os
import subprocess

SAMPLEIDS = [
    '14', '6', '87', 'F13', 'F2', 'F33', 'F37', 'FF1', 'FF104', 'FF115', 'FF13', 'FF18', 'FF20', 'FF21', 'FF23', 'FF24', 'FF27', 
    'FF3', 'FF31', 'FF34', 'FF37', 'FF39', 'FF4', 'FF43', 'FF53', 'FF56', 'FF57', 'FF58', 'FF6', 'FF62', 'FF67', 'FF71', 'FF76', 
    'FF77', 'FF78', 'FF79', 'FF80', 'FF85', 
    #'IO04',  # swapped sample
    'IO05', 'IO06', 'IO07', 'IO08', 'IO09', 'IO12', 'IO13', 'IO15', 'IO16', 'IO17', 
    'IO18', 'IO22', 'IO23', 'IO24', 'IO25', 'SC126', 'SC134', 'SC31', 'SC81', 'SC88', 'SC97', 'SC98', 
]
SAMPLEIDS_LU = [f'LU-{x}' for x in SAMPLEIDS]
SAMPLEIDS_LU_PANEL = [f'{x}_panel' for x in SAMPLEIDS_LU]
SAMPLEIDS_LU_TUMOR = [f'{x}_tumor' for x in SAMPLEIDS_LU]

BAM_TOPDIR = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/'
PANEL_REGION_PATH = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/metadata/panel_bait_design/custom/exon_intron_targetregions_merged.bed'
PANEL_REGION_EXONS_PATH = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/metadata/panel_bait_design/custom/SNUH_FIRST_Lung_Cancer_V5_exon_Regions_tracknamechanged.bed'
PANEL_REGION_INTRONS_PATH = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/metadata/panel_bait_design/custom/SNUH_FIRST_Lung_Cancer_V5_intron_Regions_tracknamechanged.bed'
WGS_TMB_PATH = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/metadata/wgs_TMB_previously_found_210312.txt'

In [4]:
sampletype_filename_map = {
    'tumor': ['tumor'], 
    'normal': ['normal'], 
    'panel': ['panel'], 
    'rna': ['RNA', 'RNA_GFP'],
}
BAM_PATHS = {sampletype: dict() for sampletype in sampletype_filename_map.keys()}

for top, dirs, files in os.walk(BAM_TOPDIR):
    for f in files:
        for sampleid in SAMPLEIDS_LU:
            for sampletype, fname_portions in sampletype_filename_map.items():
                if f in (f'{sampleid}.{fname_portion}.bam' for fname_portion in fname_portions):
                    BAM_PATHS[sampletype][sampleid] = os.path.join(top, f)

In [12]:
def get_avg_depth(bam_path):
    p = subprocess.run(
        [f'''\
            samtools depth {bam_path} |
                awk '
                NR % 50000000 == 0 {{print > "/dev/stderr"}}
                {{
                    depthsum += $3
                }}
                #NR == 100000 {{nextfile}}
                END {{
                    avg_depth = depthsum / NR
                    print avg_depth
                }}
                '
        '''], 
        shell=True,
        stdout=subprocess.PIPE,
        text=True
    )
    return float(p.stdout.strip())

In [ ]:
avg_depth_map = {'normal': dict(), 'tumor': dict()}
for sampleid in SAMPLEIDS_LU:
    for sampletype in ('normal', 'tumor'):
        bam_path = BAM_PATHS[sampletype][sampleid]
        avg_depth = get_avg_depth(bam_path)
        print(sampleid, sampletype, avg_depth)
        avg_depth_map[sampletype][sampleid] = avg_depth

1	50774414	38
1	100827320	33
1	173506737	15
1	223872109	35
2	25074158	34
2	75120868	40
2	129650798	36
2	179767032	50
2	229780011	27
3	36812477	31
3	86940944	42
3	139956102	41
3	190012369	40
4	42272116	45
4	95507932	39
4	145515487	31
5	4505118	37
5	57569967	42
5	107644901	44
5	157757719	39
6	26934179	37
6	80190143	38
6	130369591	42
7	9639716	45
7	62902444	21
7	113029013	49
8	4320761	38
8	57492390	56
8	107648733	37
9	11531404	35
9	82080682	38
9	132343932	37
10	44865508	43
10	95403573	42
11	10306751	27
11	63720451	33
11	114038354	24
12	29353025	42
12	82371145	31
12	132476401	46
13	67771180	35
14	22191907	30
14	72205710	37
15	35684565	41
15	85803188	51
16	33406803	70
17	4521536	36
17	57736653	41
18	29813824	37
19	2132932	36
19	55406943	33
20	49686692	34
22	17930705	28
X	17373852	23
X	70697878	16
X	120921885	28
Y	21801038	13


LU-14 normal 40.0436


1	50774372	33
1	100825625	26
1	173507956	44
1	223873307	42
2	25075615	33
2	75122535	19
2	129653058	44
2	179770687	42
2	229785134	30
3	36819070	26
3	86950767	34
3	139966436	28
3	190022204	37
4	42286026	40
4	95523124	47
4	145531058	34
5	4522589	49
5	57587956	44
5	107662372	30
5	157776078	36
6	26955819	31
6	80212610	42
6	130392545	40
7	9662656	34
7	62928716	34
7	113055369	38
8	4346848	41
8	57521575	46
8	107678349	49
9	11559952	24
9	82113320	37
9	132376883	40
10	44899559	48
10	95437074	43
11	10342704	31
11	63756039	29
11	114074173	30
12	29391561	37
12	82411024	35
12	132517499	23
13	67813234	37
14	22236375	28
14	72251152	40
15	35730062	34
15	85849313	44
16	33451924	49
17	4567227	30
17	57782993	39
18	29860642	50
19	2181277	23
19	55455505	34
20	49735904	36
22	17980447	32
X	17425169	18
X	70749558	22
X	120974612	21
Y	21853556	15


LU-14 tumor 36.1838


1	50781152	30
1	100831971	37
1	173549778	45
1	223910615	28
2	25115980	32
2	75182529	35
2	129701117	40
2	179828574	32
2	229843623	35
3	36882567	39
3	87001270	32
3	140011448	36
3	190128850	24
4	42446986	40
4	95785788	35
4	145804938	35
5	4804598	35
5	57872181	38
5	107953180	40
5	158082982	41
6	27259294	31
6	80501847	37
6	130680378	23
7	9958039	39
7	63209272	38
7	113333543	47
8	4618892	41
8	57780602	31
8	107932778	33
9	11803011	22
9	82366014	32
9	132623188	26
10	45154786	31
10	95719128	31
11	10631949	35
11	64047423	28
11	114361148	42
12	29609843	50
12	82627751	38
12	132839180	43
13	68039343	29
14	22438370	29
14	72447760	34
15	35939611	35
15	86047691	46
16	33653547	28
17	4773710	29
17	57944476	48
18	30026621	37
19	2340112	42
19	55615573	39
20	49895204	15
22	18168640	20
X	17584907	34
X	70901378	37
X	121114308	32


...

1	50775602	33
1	100832421	43
1	173542395	28
1	223899317	39
2	25102215	38
2	75154227	27
2	129666847	36
2	179801147	30
2	229818782	23
3	36855935	26
3	86977336	37
3	139998216	40
3	190032373	23
4	42317461	25
4	95658544	32
4	145672890	39
5	4663750	24
5	57726541	30
5	107789912	32
5	157910586	27
6	27142770	32
6	80376220	30
6	130529257	31
7	9795544	41
7	63058192	28
7	113178192	33
8	4470573	26
8	57624127	32
8	107776513	33
9	11639919	33
9	82214190	36
9	132478952	20
10	45018471	25
10	95583302	29
11	10501357	42
11	63946814	45
11	114268893	37
12	29588006	29
12	82610226	28
12	132816867	12
13	68011259	31
14	22415407	33
14	72424400	24
15	35904169	41
15	86013102	38
16	33617724	53
17	4745706	23
17	57901110	28
18	29979893	21
19	2295331	34
19	55564322	34
20	49842409	33
22	18081570	35
X	17509638	11
X	70841268	15
X	121060213	15
Y	21944591	8


LU-FF58 normal 33.4403


1	50776027	20
1	100833769	28
1	173546093	34
1	223902596	35
2	25107005	29
2	75160372	30
2	129674366	35
2	179809474	32
2	229827744	29
3	36866971	32
3	86987917	25
3	140009781	28
3	190043456	29
4	42329424	32
4	95673280	36
4	145687298	24
5	4677797	30
5	57739912	30
5	107803058	28
5	157923228	46
6	27155191	30
6	80388674	30
6	130542115	25
7	9809041	36
7	63071676	28
7	113193193	25
8	4484974	40
8	57639707	36
8	107792918	35
9	11656630	38
9	82233144	36
9	132498818	34
10	45037607	48
10	95604325	36
11	10523269	29
11	63970409	22
11	114292903	40
12	29609006	34
12	82630921	21
12	132837993	28
13	68032439	32
14	22436868	24
14	72446217	33
15	35926818	35
15	86036154	35
16	33641575	50
17	4769800	30
17	57925964	27
18	30004257	34
19	2319327	26
19	55587966	36
20	49866569	36
22	18106221	20
X	17537333	17
X	70869118	10
X	121089687	24
Y	21975019	10


LU-FF58 tumor 32.9969


1	50778659	35
1	100829283	33
1	173508436	28
1	223867007	34
2	25075253	24
2	75122213	35
2	129638642	21
2	179769102	34
2	229782403	41
3	36824726	40
3	86951525	31
3	139967105	38
3	190085245	43
4	42410181	34
4	95768619	37
4	145774682	40
5	4769561	39
5	57832627	30
5	107908820	31
5	158014589	31
6	27190149	35
6	80436710	24
6	130614893	46
7	9881695	35
7	63136698	35
7	113261211	50
8	4534304	31
8	57779640	33
8	107939384	40
9	11801844	44
9	82396000	33
9	132659185	28
10	45187438	27
10	95742098	25
11	10642557	25
11	64057962	19
11	114372464	32
12	29631667	35
12	82646396	31
12	132851230	38
13	68034758	33
14	22438589	28
14	72445638	28
15	35924527	37
15	86045966	30
16	33659895	31
17	4802599	30
17	57966235	30
18	30040356	42
19	2348884	39
19	55635119	21
20	49915443	33
22	18155962	26
X	17557038	14
